## 🔗 Montar Google Drive para acceder a archivos del Scheduling Problem

Este bloque conecta Google Colab con Google Drive. Es necesario para poder cargar o guardar archivos relacionados con el algoritmo genético aplicado al problema de generación de horarios escolares. Asegura acceso a datos como los cromosomas iniciales, los logs de evolución, y los archivos con los mejores individuos encontrados.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 📂 Cargar datasets del Scheduling Problem desde Google Drive

Este bloque carga los datasets principales que definen las condiciones del problema de generación de horarios:

- `docentes.csv`: contiene los profesores y las materias que dictan.
- `materias.csv`: define cada materia con su respectivo nivel y carga horaria.
- `paralelos.csv`: lista los cursos y aulas por cada nivel educativo.
- `horarios.csv`: define los rangos de tiempo disponibles para la asignación.

Estos datos son fundamentales para la construcción de cromosomas válidos y la evaluación del fitness en cada generación del algoritmo genético.


In [ ]:
import pandas as pd
import random

ruta = '/content/drive/MyDrive/Tesis_MSDS'
# Cargar los datos
docentes = pd.read_csv(ruta+"/docentes.csv")
materias = pd.read_csv(ruta+"/materias.csv")
paralelos = pd.read_csv(ruta+"/paralelos.csv")
horarios = pd.read_csv(ruta+"/horarios.csv")

## 🧹 Preprocesamiento: Unificación de docentes por asignatura especializada

Este bloque aplica reglas de unificación de nombres de docentes para ciertas materias específicas, con el objetivo de mantener consistencia en la asignación horaria durante el proceso evolutivo:

- **Inglés desde 7mo EGB en adelante**: Se consolidan los docentes `DAISY HERRERA`, `ELIZABETH PUGA` y `SIMONE AYALA` bajo el identificador común `INGLES ESPECIAL`.
- **Educación Física desde 3ro EGB**: Se unifican `NANCY LINCANGO`, `CARLOS MORALES` e `IVÁN MALDONADO` como `EF ESPECIAL`.

Esta estandarización permite asegurar que estas materias sean asignadas de forma sincronizada entre paralelos del mismo nivel durante la construcción de cromosomas y aplicación de restricciones duras.


In [ ]:
#Unificar docentes de inglés a partir de 7mo EGB
docentes=docentes.drop(docentes[docentes['Nombres']=='ELIZABETH PUGA'].index)
docentes=docentes.drop(docentes[docentes['Nombres']=='SIMONE AYALA'].index)
docentes["Nombres"] = docentes["Nombres"].replace("DAISY HERRERA", "INGLES ESPECIAL")

#Unificar docentes de Educación Física a partir de 3ro EGB
docentes=docentes.drop(docentes[docentes['Nombres']=='CARLOS MORALES'].index)
docentes=docentes.drop(docentes[docentes['Nombres']=='IVÁN MALDONADO'].index)
docentes["Nombres"] = docentes["Nombres"].replace("NANCY LINCANGO", "EF ESPECIAL")

docentes.reset_index()

## 🧾 Normalización de códigos de asignaturas por docente

Este bloque transforma la columna `Codigo` del dataset de docentes, que originalmente contiene conjuntos de códigos como strings (`{'MAT_3EGB', 'LEN_3EGB'}`), en un formato plano y legible para el modelo genético.

### Transformaciones aplicadas:
- 🔍 **Limpieza**: Se eliminan caracteres como `{}`, comillas simples y espacios extra.
- 🔄 **Explosión de filas**: Cada código de materia impartido por un docente pasa a una fila separada.

Esto es esencial para permitir una asignación precisa durante la generación y evaluación de cromosomas, ya que cada código representa una materia que debe ser distribuida en el horario.


In [ ]:
df=docentes.copy()
# Limpiar la columna 'Codigo': eliminar llaves y comillas simples
df["Codigo"] = df["Codigo"].str.replace(r"[{}']", "", regex=True)

# Separar los códigos por coma y expandir en filas individuales
docentes = df.assign(Codigo=df["Codigo"].str.split(", ")).explode("Codigo")

docentes.head(5)

,index,Nombres,Cargo,Materia,Codigo
0,0,MALENA OÑA,Tiempo completo,LENGUAJE,LEN_1EGB
1,1,ALISON PLAZA,Tiempo completo,LECTORES,LEC_1EGB
1,1,ALISON PLAZA,Tiempo completo,LECTORES,LEC_2EGB
1,1,ALISON PLAZA,Tiempo completo,LECTORES,LEC_3EGB
2,2,SOFÍA YÉPEZ,Tiempo completo,LENGUAJE,LEN_3EGB
2,2,SOFÍA YÉPEZ,Tiempo completo,LENGUAJE,LEN_2EGB
3,3,ROSITA ALBUJA,Tiempo completo,LENGUAJE,LEN_5EGB
3,3,ROSITA ALBUJA,Tiempo completo,LENGUAJE,LEN_4EGB
4,4,NARCISA MUÑOZ,Tiempo completo,LENGUAJE,LEN_6EGB
4,4,NARCISA MUÑOZ,Tiempo completo,LENGUAJE,LEN_7EGB


## 🧠 Construcción de diccionario profesor ↔ código de materia

Este bloque crea un diccionario que asocia directamente cada código de materia con el nombre del docente correspondiente.

### Objetivo:
Permitir una consulta eficiente del docente responsable de una materia específica, lo cual es clave para:
- Asignar correctamente los docentes al construir cada cromosoma.
- Aplicar restricciones como la no duplicación de docentes en simultáneo.

### Transformaciones aplicadas:
- Limpieza de strings tipo `{‘LEN_3EGB’, ‘LEN_2EGB’}` para extraer códigos individuales.
- Creación del diccionario `dic_profesores` en formato:  
  ```python
  {'LEN_3EGB': 'MALENA OÑA', 'EST_10EGB': 'SANTIAGO CALLE', ...}


In [ ]:
import pandas as pd
df=docentes.copy()
# Carga del archivo CSV
df = docentes

# Identifica las columnas
col_codigo = 'Codigo'
col_profesor = 'Nombres'

# Diccionario resultado
dic_profesores = {}

for _, row in df.iterrows():
    raw = row[col_codigo]
    if pd.isna(raw):
        continue

    # Convertimos a cadena y eliminamos espacios exteriores
    s = str(raw).strip()

    # Si empieza por '{' o '[' acabamos quitando delimitadores
    if s.startswith('{') and s.endswith('}'):
        # Quitamos las llaves
        s = s[1:-1]
    elif s.startswith('[') and s.endswith(']'):
        # Quitamos los corchetes
        s = s[1:-1]

    # Ahora s es algo como "'LEC_1EGB', 'LEC_3EGB', 'LEC_2EGB'"
    # Partimos por comas y limpiamos comillas
    codes = []
    for part in s.split(','):
        part = part.strip().strip("'\"")  # quita comillas simples o dobles
        if part:
            codes.append(part)

    # Asociamos cada código con el profesor
    for code in codes:
        dic_profesores[code] = row[col_profesor]

# Ejemplo de uso:
for ejemplo in ['LEN_3EGB', 'PEN_10EGB', 'EST_10EGB']:
    prof = dic_profesores.get(ejemplo, 'No encontrado')
    print(f"Profesor de {ejemplo}: {prof}")

Profesor de LEN_3EGB: SOFÍA YÉPEZ
Profesor de PEN_10EGB: MIREYA BRITO
Profesor de EST_10EGB: JONATHAN CASTRO


In [ ]:
dic_profesores['LEN_1EGB']

'MALENA OÑA'

## 📅 Definición de parámetros y restricciones de horario

Este bloque define las funciones auxiliares y restricciones necesarias para construir cromosomas válidos, tomando en cuenta reglas propias del calendario escolar del colegio.

### Componentes definidos:

- `dias_semana`: Lista base de días laborables.
- `obtener_materias_por_nivel(nivel)`: Devuelve la lista de códigos de materia correspondientes a un nivel educativo específico (por ejemplo, 8EGB).
- `obtener_docente_por_materia_codigo(codigo)`: Consulta el docente responsable de un código de materia usando el diccionario `dic_profesores`.
- `es_horario_valido(nivel, dia, hora)`: Verifica si un horario específico es válido para un determinado nivel. Esta función incorpora:
  - Restricciones por **receso** y **almuerzo**.
  - Bloqueos por eventos institucionales como **Evalev** y **Clubs extracurriculares**.
  - Diferenciación por nivel educativo y día de la semana.

Estas validaciones son fundamentales para asegurar que las asignaciones generadas durante el algoritmo evolutivo cumplan con las restricciones duras del problema de Scheduling.

In [ ]:
# Lista de días de la semana
dias_semana = ["Lunes", "Martes", "Miércoles", "Jueves", "Viernes"]

# Filtrar materias según nivel
def obtener_materias_por_nivel(nivel):
    return materias[materias["Nivel"] == nivel]["Código"].tolist()

# Obtener docente que imparte la materia
def obtener_docente_por_materia_codigo(codigo_materia):
    return dic_profesores.get(codigo_materia, None)

def es_horario_valido(nivel, dia, hora):
    # Receso
    if nivel in ["1EGB", "2EGB", "3EGB"] and hora == 5:
        return False
    if nivel not in ["1EGB", "2EGB", "3EGB"] and hora == 6:
        return False

    # Almuerzo
    if nivel in ["1EGB", "2EGB", "3EGB", "4EGB", "5EGB", "6EGB", "7EGB"] and hora in [11, 12]:
        return False
    if nivel in ["8EGB", "9EGB", "10EGB", "1BGU", "2BGU", "3BGU"] and hora in [13, 14]:
        return False

    # Evalev
    if dia == "Miércoles" and hora in [1, 2]:
        return False

    # Clubs
    if nivel == "1EGB" and ((dia == "Lunes" and hora in [15, 16]) or (dia == "Jueves" and hora in [13, 14])):
        return False
    if nivel in ["2EGB", "3EGB", "4EGB", "5EGB", "6EGB", "7EGB"] and dia in ["Martes", "Jueves"] and hora in [15, 16]:
        return False
    if nivel in ["8EGB", "9EGB", "10EGB", "1BGU", "2BGU", "3BGU"] and dia in ["Miércoles", "Viernes"] and hora in [15, 16]:
        return False

    return True


## 🧮 Función de ordenamiento de cromosomas por nivel, aula, día y hora

Esta función organiza los genes de un cromosoma en un orden lógico y jerárquico para facilitar su análisis, visualización y depuración.

### Criterios de ordenamiento aplicados:
1. **Nivel educativo** en el orden oficial: de 1EGB hasta 3BGU.
2. **Paralelo** en orden alfabético.
3. **Día de la semana**: de lunes a viernes.
4. **Hora**: en orden creciente (desde la primera hora del día).

Esto garantiza que el cromosoma tenga una estructura consistente a lo largo del proceso evolutivo, lo cual es útil tanto para el monitoreo como para asegurar una correcta interpretación por parte de los operadores genéticos.

> 💡 Esta función no altera el contenido del cromosoma, solo su orden para facilitar operaciones posteriores.


In [ ]:
def ordenar_cromosoma(cromosoma):
    """
    Ordena la lista de genes por:
      1) Nivel, en el orden 1EGB→2EGB→…→10EGB→1BGU→2BGU→3BGU
      2) Paralelo (alfabéticamente)
      3) Día de la semana (Lunes→Viernes)
      4) Hora (ascendente)
    """
    # 1) Definir orden explícito de niveles
    niveles_orden = [
        "1EGB","2EGB","3EGB","4EGB","5EGB","6EGB","7EGB",
        "8EGB","9EGB","10EGB",
        "1BGU","2BGU","3BGU"
    ]
    idx_nivel = {niv: i for i, niv in enumerate(niveles_orden)}

    # 2) Orden de días
    dias_orden = {dia: i for i, dia in enumerate(["Lunes","Martes","Miércoles","Jueves","Viernes"])}

    # 3) Sort usando key múltiple
    cromosoma_ordenado = sorted(
        cromosoma,
        key=lambda gen: (
            idx_nivel.get(gen[0], 999),  # Nivel (por si hay alguno inesperado)
            gen[1],                       # Paralelo
            dias_orden[gen[2]],           # Día según dias_orden
            gen[3]                        # Hora
        )
    )
    return cromosoma_ordenado


## ⏰ Asignación sincronizada de materias en múltiples aulas por nivel

Esta función permite asignar una misma materia (`codigo_materia`) a múltiples aulas simultáneamente dentro de uno o más niveles educativos. Se asegura de que todas las asignaciones compartan el mismo día y hora, respetando las restricciones del modelo.

### Parámetros:
- `cromosoma`: lista actual de genes a modificar.
- `niveles_target`: niveles educativos involucrados en la sincronización (por ejemplo, `["7EGB", "8EGB", "9EGB"]`).
- `codigo_materia`: código único de la materia a asignar (por ejemplo, `"ING_7EGB"`).
- `cantidad_horas`: número total de bloques a asignar para esta materia.
- `en_parejas`: si `True`, se asignan **bloques dobles continuos** (por ejemplo, para materias con sesiones extendidas).

### Características clave:
- Utiliza `es_horario_valido` para verificar que los bloques estén disponibles y no caigan en recesos, almuerzos o eventos institucionales.
- Aplica verificación cruzada para evitar colisiones de horario en todas las aulas y niveles involucrados.
- El resultado es un cromosoma actualizado con las nuevas asignaciones sincronizadas.

Esta función es fundamental para materias como **Inglés Especial** o **Educación Física**, que deben dictarse en paralelo entre varias aulas del mismo nivel.


In [ ]:
def asignar_misma_hora_multiple_aulas_actualizado(cromosoma, niveles_target, codigo_materia, cantidad_horas, en_parejas=False):
    aulas_por_nivel = {
        nivel: paralelos.loc[paralelos["Nivel"] == nivel, "Nombre_paralelo"].tolist()
        for nivel in niveles_target
    }

    posibles_slots = []
    for dia in dias_semana:
        for h in range(1, 17 if not en_parejas else 16):
            if en_parejas and not all(es_horario_valido(n, dia, h) and es_horario_valido(n, dia, h+1) for n in niveles_target):
                continue
            if not en_parejas and not all(es_horario_valido(n, dia, h) for n in niveles_target):
                continue
            posibles_slots.append((dia, h))

    random.shuffle(posibles_slots)
    horas_asignadas = 0

    for dia, hora in posibles_slots:
        ocupado = False
        for nivel in niveles_target:
            for aula in aulas_por_nivel[nivel]:
                if any((g[0] == nivel and g[1] == aula and g[2] == dia and g[3] == hora) or
                       (en_parejas and g[0] == nivel and g[1] == aula and g[2] == dia and g[3] == hora+1)
                       for g in cromosoma):
                    ocupado = True
                    break
        if ocupado:
            continue

        for nivel in niveles_target:
            for aula in aulas_por_nivel[nivel]:
                docente = obtener_docente_por_materia_codigo(codigo_materia)
                cromosoma.append((nivel, aula, dia, hora, codigo_materia, docente))
                if en_parejas:
                    cromosoma.append((nivel, aula, dia, hora+1, codigo_materia, docente))

        horas_asignadas += 2 if en_parejas else 1
        if horas_asignadas >= cantidad_horas:
            break

    return cromosoma



## 🧬 Generación de cromosoma inicial con bloques sincronizados y asignaciones estructuradas

Esta función construye un cromosoma válido para el problema de Scheduling Educativo. Cada cromosoma representa un horario completo con todas las asignaciones posibles de materias, aulas, días y docentes, cumpliendo con restricciones duras como horarios válidos y franjas especiales por nivel.

### Fases principales de la generación:

#### 1️⃣ **Asignaciones especiales sincronizadas por nivel**
- Materias como **INGLES ESPECIAL**, **ESTUDIOS SOCIALES**, **DESARROLLO PERSONAL** o **CIUDADANÍA** se asignan en bloques comunes para todas las aulas de un mismo nivel, usando la función `asignar_misma_hora_multiple_aulas_actualizado`.

#### 2️⃣ **Asignación por pares preferidos**
- Se crean bloques de clases en pares, respetando combinaciones pedagógicas predefinidas como:
  - `DER` ↔ `COM`
  - `SPE` ↔ `INV`
  - `SAL` ↔ `ECO`
  - `EXP` ↔ `RAZ` / `CAL`
  - `DAN` ↔ `MUS`
- Estos pares se asignan en bloques contiguos (slots dobles) en días y horas válidas.

#### 3️⃣ **Relleno de ranuras restantes**
- Se llena el resto del horario con las materias disponibles según la carga horaria restante, validando siempre:
  - Que el horario sea válido (`es_horario_valido`)
  - Que no exista una asignación previa en el mismo bloque

#### 4️⃣ **Orden final**
- El cromosoma generado se ordena al final por nivel, paralelo, día y hora para mantener consistencia y facilitar el análisis visual y evolutivo.

> ⚠️ Cada cromosoma generado es diferente, ya que intervienen decisiones aleatorias (`random.shuffle`, `random.choice`) tanto en la selección de materias como en la ubicación de horarios, lo que aporta diversidad genética inicial a la población.


In [ ]:
import random

def generar_cromosoma():
    cromosoma = []
    niveles = paralelos["Nivel"].unique()

    # 1. Asignaciones comunes por nivel
    niveles_ingles = ["7EGB", "8EGB", "9EGB", "10EGB", "1BGU", "2BGU", "3BGU"]
    for nivel in niveles_ingles:
        cod = f"ING_{nivel}"
        cromosoma = asignar_misma_hora_multiple_aulas_actualizado(cromosoma, [nivel], cod, cantidad_horas=6, en_parejas=True)

    cromosoma = asignar_misma_hora_multiple_aulas_actualizado(cromosoma, ["3BGU"], "EST_3BGU", cantidad_horas=2, en_parejas=True)
    cromosoma = asignar_misma_hora_multiple_aulas_actualizado(cromosoma, ["3BGU"], "DES_3BGU", cantidad_horas=2, en_parejas=True)
    cromosoma = asignar_misma_hora_multiple_aulas_actualizado(cromosoma, ["1BGU"], "CIU_1BGU", cantidad_horas=1, en_parejas=False)
    cromosoma = asignar_misma_hora_multiple_aulas_actualizado(cromosoma, ["2BGU"], "CIU_2BGU", cantidad_horas=1, en_parejas=False)

    # 2. Pares preferidos
    pares_pref = {
        "DER": "COM",
        "SPE": "INV",
        "SAL": "ECO",
        "EXP": ["RAZ", "CAL"],
        "DAN": "MUS"
    }

    for nivel in niveles:
        aulas = paralelos.loc[paralelos["Nivel"] == nivel, "Nombre_paralelo"]
        materias_nivel = materias[materias["Nivel"] == nivel]

        for aula in aulas:
            bolsa = []
            for _, row in materias_nivel.iterrows():
                bolsa += [row["Código"]] * int(row["Horas"])

            bolsa = [c for c in bolsa if not any(g[0] == nivel and g[1] == aula and g[4] == c for g in cromosoma)]

            # Buscar ranuras dobles disponibles
            slots_pares = []
            for dia in dias_semana:
                horas_validas = sorted(h for h in range(1, 16)
                    if es_horario_valido(nivel, dia, h)
                    and es_horario_valido(nivel, dia, h+1)
                    and not any(g[0] == nivel and g[1] == aula and g[2] == dia and g[3] in [h, h+1] for g in cromosoma))
                for h in horas_validas:
                    if h + 1 in horas_validas:
                        slots_pares.append((dia, h))

            lista_pares = []
            claves = list(pares_pref.keys())
            random.shuffle(claves)
            for pref in claves:
                pareja = pares_pref[pref]
                cods1 = [c for c in bolsa if c.startswith(pref)]
                if isinstance(pareja, list):
                    cods2 = [c for c in bolsa if any(c.startswith(p) for p in pareja)]
                else:
                    cods2 = [c for c in bolsa if c.startswith(pareja)]
                num = min(len(cods1), len(cods2))

                for _ in range(num):
                    c1 = cods1.pop(); bolsa.remove(c1)
                    if isinstance(pareja, list):
                        c2 = next(c for c in cods2 if any(c.startswith(p) for p in pareja))
                        cods2.remove(c2)
                    else:
                        c2 = cods2.pop()
                    bolsa.remove(c2)

                    # 🔁 Orden aleatorio de la pareja
                    pareja_ordenada = [c1, c2]
                    random.shuffle(pareja_ordenada)
                    lista_pares.append(tuple(pareja_ordenada))


            random.shuffle(lista_pares)
            random.shuffle(slots_pares)
            for (dia, h), (c1, c2) in zip(slots_pares, lista_pares):
                d1 = obtener_docente_por_materia_codigo(c1)
                d2 = obtener_docente_por_materia_codigo(c2)
                cromosoma.append((nivel, aula, dia, h, c1, d1))
                cromosoma.append((nivel, aula, dia, h+1, c2, d2))

            # 3. Relleno final
            for dia in dias_semana:
                for hora in range(1, 17):
                    if not es_horario_valido(nivel, dia, hora):
                        continue
                    if any(g[0] == nivel and g[1] == aula and g[2] == dia and g[3] == hora for g in cromosoma):
                        continue
                    if bolsa:
                        idx = random.randrange(len(bolsa))
                        c = bolsa.pop(idx)
                        d = obtener_docente_por_materia_codigo(c)
                        cromosoma.append((nivel, aula, dia, hora, c, d))

    return ordenar_cromosoma(cromosoma)


In [ ]:
# Generar cromosoma y mostrar validación
cromosoma_ejemplo = generar_cromosoma()
print(f"Total de genes generados: {len(cromosoma_ejemplo)}")

# Mostrar los primeros 5 genes
for gen in cromosoma_ejemplo[:]:
    print(gen)

Total de genes generados: 2832
('1EGB', 'Ejemplares', 'Lunes', 1, 'EST_1EGB', 'SANTIAGO CALLE')
('1EGB', 'Ejemplares', 'Lunes', 2, 'MAT_1EGB', 'PAULINA ÑACATO')
('1EGB', 'Ejemplares', 'Lunes', 3, 'LEC_1EGB', 'ALISON PLAZA')
('1EGB', 'Ejemplares', 'Lunes', 4, 'EST_1EGB', 'SANTIAGO CALLE')
('1EGB', 'Ejemplares', 'Lunes', 6, 'INV_1EGB', 'KARLA LAZCANO')
('1EGB', 'Ejemplares', 'Lunes', 7, 'SPE_1EGB', 'CHRISTIAN SOLANO')
('1EGB', 'Ejemplares', 'Lunes', 7, 'SAL_1EGB', 'VANESSA ESPÍN')
('1EGB', 'Ejemplares', 'Lunes', 8, 'ECO_1EGB', 'MARJURIE JÁCOME')
('1EGB', 'Ejemplares', 'Lunes', 9, 'LEN_1EGB', 'MALENA OÑA')
('1EGB', 'Ejemplares', 'Lunes', 10, 'LEN_1EGB', 'MALENA OÑA')
('1EGB', 'Ejemplares', 'Lunes', 13, 'PEN_1EGB', 'FERNANDA CAÑIZARES')
('1EGB', 'Ejemplares', 'Lunes', 14, 'MAT_1EGB', 'PAULINA ÑACATO')
('1EGB', 'Ejemplares', 'Martes', 1, 'LEN_1EGB', 'MALENA OÑA')
('1EGB', 'Ejemplares', 'Martes', 2, 'NAT_1EGB', 'MARIANA HIDALGO')
('1EGB', 'Ejemplares', 'Martes', 3, 'ING_1EGB', 'MARCIA HERRER

## 📊 Conteo de asignaciones por aula dentro de un cromosoma

Esta función permite auditar cuántas asignaciones horarias tiene cada aula en un cromosoma generado. Se utiliza para verificar el equilibrio en la distribución de bloques por paralelo y nivel.

### ¿Qué hace exactamente?
- Recorre todos los genes del cromosoma.
- Agrupa por `(Nivel, Aula)`.
- Devuelve un `Counter` con el número total de asignaciones por aula.

### Aplicaciones:
- Validar si las aulas están cumpliendo con la carga horaria total.
- Detectar asignaciones insuficientes o excesivas por curso.
- Monitorear la convergencia estructural del algoritmo evolutivo.

> 💡 Esta función es útil tanto para diagnósticos en la población inicial como para evaluar consistencia al final del proceso evolutivo.


In [ ]:
from collections import Counter

def contar_asignaciones_por_aula(cromosoma):
    """
    Devuelve un Counter cuyas claves son tuplas (Nivel, Aula)
    y los valores el número de genes asignados a esa aula.
    """
    contador = Counter((nivel, aula) for nivel, aula, *_ in cromosoma)
    return contador

# Uso:
ctr = contar_asignaciones_por_aula(cromosoma_ejemplo)
for (nivel, aula), num_genes in ctr.items():
    print(f"{nivel} • {aula}: {num_genes}")


1EGB • Ejemplares: 59
1EGB • Entusiastas: 59
1EGB • Estudiosos: 59
1EGB • Exitosos: 59
2EGB • Afectuosos: 59
2EGB • Amables: 59
2EGB • Amigables: 59
2EGB • Amistosos: 59
3EGB • Colaboradores: 59
3EGB • Conciliadores: 59
3EGB • Cordiales: 59
3EGB • Creativos: 59
4EGB • Optimistas: 59
4EGB • Ordenados: 59
4EGB • Organizados: 59
4EGB • Originales: 59
5EGB • Admirables: 59
5EGB • Afectivos: 59
5EGB • Aplicados: 59
5EGB • Atentos: 59
6EGB • Ecuánimes: 59
6EGB • Emprendedores: 59
6EGB • Excelentes: 59
6EGB • Expresivos: 59
7EGB • Reflexivos: 59
7EGB • Resilientes: 59
7EGB • Respetuosos: 59
7EGB • Responsables: 59
8EGB • Decididos: 59
8EGB • Diligentes: 59
8EGB • Dinámicos: 59
8EGB • Disciplinados: 59
9EGB • Altruistas: 59
9EGB • Asertivos: 59
9EGB • Audaces: 59
9EGB • Auténticos: 59
10EGB • Pacifistas: 59
10EGB • Perspicaces: 59
10EGB • Positivos: 59
1BGU • Confiables: 59
1BGU • Integros: 59
1BGU • Leales: 59
2BGU • Honestos: 59
2BGU • Justos: 59
2BGU • Proactivos: 59
3BGU • Empáticos: 59
3B

## 🧾 Conteo de horas asignadas por materia en cada aula

Esta función convierte un cromosoma en una tabla estructurada que muestra cuántas horas ha sido asignada cada materia en cada paralelo.

### Funcionalidad:
- Transforma el cromosoma (lista de tuplas) en un `DataFrame`.
- Agrupa por `(Nivel, Aula, Código_materia)` para contar cuántas veces aparece cada materia.
- Devuelve un `DataFrame` ordenado que permite comparar lo asignado con lo esperado.

### Aplicaciones:
- Verificar si las materias cumplen con su carga horaria oficial.
- Identificar desbalances o errores de asignación por curso.
- Monitorear la coherencia interna de cromosomas generados por el algoritmo evolutivo.

> ✅ Este resumen es especialmente útil al final de la evolución para validar la factibilidad de los horarios generados.


In [ ]:
import pandas as pd

def contar_horas_por_materia(cromosoma):
    """
    Recibe:
        cromosoma: lista de tuplas
           (Nivel, Aula, Día, Hora, Código_materia, Docente)
    Devuelve un DataFrame con el número de horas asignadas por materia en cada aula.
    """
    # 1) Convertir la lista de genes en un DataFrame
    df = pd.DataFrame(
        cromosoma,
        columns=["Nivel", "Aula", "Día", "Hora", "Código_materia", "Docente"]
    )

    # 2) Agrupar por Nivel, Aula y Código_materia, y contar ocurrencias (horas)
    resumen = (
        df
        .groupby(["Nivel", "Aula", "Código_materia"])
        .size()
        .reset_index(name="Horas_asignadas")
        .sort_values(["Nivel", "Aula", "Código_materia"])
    )

    return resumen

# Ejemplo de uso:
df_horas = contar_horas_por_materia(cromosoma_ejemplo)
df_horas


,Nivel,Aula,Código_materia,Horas_asignadas
0,10EGB,Pacifistas,BIO_10EGB,3
1,10EGB,Pacifistas,COM_10EGB,1
2,10EGB,Pacifistas,DAN_10EGB,1
3,10EGB,Pacifistas,DER_10EGB,1
4,10EGB,Pacifistas,DES_10EGB,2
...,...,...,...,...
1095,9EGB,Auténticos,QUI_9EGB,6
1096,9EGB,Auténticos,ROB_9EGB,2
1097,9EGB,Auténticos,SAL_9EGB,1
1098,9EGB,Auténticos,SOC_9EGB,3


## 🔍 Verificación de materias con carga horaria incompleta por aula

Esta función compara la cantidad de horas **asignadas** vs. **requeridas** para cada materia en cada aula, identificando aquellas que no cumplen con su carga horaria establecida.

### ¿Qué hace esta función?
1. Convierte el cromosoma en un `DataFrame` para análisis tabular.
2. Calcula cuántas horas ha sido asignada cada materia por curso y paralelo.
3. Replica la estructura esperada de materias por aula a partir de los archivos `materias.csv` y `paralelos.csv`.
4. Compara lo asignado con lo requerido.
5. Devuelve un `DataFrame` con aquellas materias que tienen **horas faltantes**.

### Aplicaciones:
- Diagnóstico inmediato de **inconsistencias estructurales** en un cromosoma.
- Verificación post-evolutiva para asegurar **factibilidad horaria**.
- Guía para penalizaciones dentro de la función fitness.

> ✅ Este paso es crítico para garantizar que todas las materias se impartan el número adecuado de veces según lo planificado por el colegio.


In [ ]:
import pandas as pd

def verificar_materias_faltantes(cromosoma, materias_df, paralelos_df):
    """
    Compara la cantidad de horas asignadas vs requeridas para cada materia en cada aula.

    Retorna un DataFrame con las materias faltantes por aula.
    """
    # 1. Convertir cromosoma a DataFrame
    df_crom = pd.DataFrame(cromosoma, columns=["Nivel", "Aula", "Día", "Hora", "Código", "Docente"])

    # 2. Contar horas asignadas por materia por aula
    asignadas = (
        df_crom.groupby(["Nivel", "Aula", "Código"])
        .size()
        .reset_index(name="Horas_asignadas")
    )

    # 3. Expandir requerimientos por aula
    requeridas = pd.merge(
        paralelos_df[["Nivel", "Nombre_paralelo"]],
        materias_df,
        on="Nivel",
        how="left"
    ).rename(columns={"Nombre_paralelo": "Aula", "Código": "Código", "Horas": "Horas_requeridas"})

    # 4. Unir ambas tablas
    comparacion = pd.merge(
        requeridas,
        asignadas,
        how="left",
        on=["Nivel", "Aula", "Código"]
    )

    comparacion["Horas_asignadas"] = comparacion["Horas_asignadas"].fillna(0).astype(int)
    comparacion["Faltan"] = comparacion["Horas_requeridas"] - comparacion["Horas_asignadas"]

    # 5. Filtrar las que faltan
    faltantes = comparacion[comparacion["Faltan"] > 0].sort_values(["Nivel", "Aula", "Código"])

    return faltantes


In [ ]:
faltantes_df = verificar_materias_faltantes(cromosoma_ejemplo, materias, paralelos)
display(faltantes_df)


,Nivel,Aula,Materias,Código,Horas_requeridas,Horas_asignadas,Faltan


## 🧱 Evaluación de restricciones duras en el cromosoma

Esta función calcula una penalización acumulativa que representa cuántas **restricciones duras** se han violado en un cromosoma generado. Estas restricciones son consideradas **obligatorias** para la validez del horario.

### 📋 Restricciones aplicadas:

1. **🔗 Continuidad de sesiones**  
   - Penaliza si una misma materia se asigna el mismo día pero con un hueco mayor a 2 horas (solo si tiene más de 1 hora total en la semana).

2. **📚 Exceso de bloques continuos**  
   - Si una materia aparece más de 3 veces de forma consecutiva el mismo día, se penaliza (excepto para casos pedagógicos específicos como MAT/LEN en 1EGB o MAT en 2EGB).

3. **🚫 Cruces de docentes**  
   - Se penaliza fuertemente si un docente aparece asignado a más de un paralelo en el mismo bloque horario.
   - Se consideran **excepciones específicas** por nombre y materia (por ejemplo, docentes de materias especiales como `EF ESPECIAL`, `INGLES ESPECIAL` o asignaturas de ciudadanía en BGU).

### ⚖️ Penalización:
- Penalizaciones pequeñas (0.1) para continuidad.
- Moderadas (2) para excesos seguidos.
- Fuertes (20×n) por cruces de docentes.

> Esta función se usa como parte del componente **fitness** en el algoritmo genético, y su correcta implementación garantiza la **factibilidad estructural mínima** del horario.


In [ ]:
from collections import defaultdict

# Asumimos que este diccionario ya fue creado previamente
horas_por_materia = dict(zip(materias["Código"], materias["Horas"]))

def evaluar_restricciones_duras(cromosoma):
    penalizaciones = 0

    sesiones_por_materia = defaultdict(lambda: defaultdict(int))  # materia -> día -> cantidad
    sesiones_por_hora = defaultdict(list)  # (día, hora) -> lista de materias

    materia_actual = None
    dia_actual = None
    hora_anterior = None

    # 1. Verificación de continuidad sin ordenar, aprovechando el orden del cromosoma
    for nivel, aula, dia, hora, materia, docente in cromosoma:
        dia = dia
        hora = int(hora)

        # Verificación de continuidad
        if materia != materia_actual or dia != dia_actual:
            # reiniciamos el seguimiento
            materia_actual = materia
            dia_actual = dia
            hora_anterior = hora
        else:
            # comparación directa, sin ordenar
            if horas_por_materia.get(materia, 0) > 1 and (hora - hora_anterior > 2):
                penalizaciones += 0.1

                # LOG de trazabilidad
                #print(f"[SIN CONTINUIDAD] {materia} en {nivel}-{aula} con {docente} el día {dia} entre hora {hora_anterior} y {hora}")

            hora_anterior = hora

        # Recuento para otras restricciones
        sesiones_por_materia[materia][dia] += 1
        sesiones_por_hora[(dia, hora)].append(materia)

    # 2. Restricción: más de 3 horas continuas de una misma materia
    materia_anterior = None
    dia_anterior = None
    racha_continua = 1

    for i in range(1, len(cromosoma)):
        _, _, dia_act, hora_act, materia_act, _ = cromosoma[i]
        _, _, dia_ant, hora_ant, materia_ant, _ = cromosoma[i - 1]

        if nivel == "1EGB" and (materia == "MAT_1EGB" or materia == "LEN_1EGB"):
          continue

        if nivel == "2EGB" and materia == "MAT_2EGB":
          continue

        if materia_act == materia_ant and dia_act == dia_ant and int(hora_act) == int(hora_ant) + 1:
            racha_continua += 1
            if racha_continua > 3:
                penalizaciones += 2  # penalización moderada por exceso de continuidad
        else:
            racha_continua = 1  # reiniciar contador si cambia la materia o no es consecutiva

    # 3. Cruces de docentes en el mismo horario (con excepciones por nombre y cargo)
    docentes_en_tiempo = defaultdict(lambda: defaultdict(set))  # (día, hora) -> docente -> set de paralelos
    cargos_docentes = dict(zip(docentes["Nombres"], docentes["Cargo"]))  # asume que 'docentes' está cargado

    for nivel, paralelo, dia, hora, materia, docente in cromosoma:
        # Excepciones:
        if docente in {"EF ESPECIAL", "INGLES ESPECIAL"}:
            continue
        if docente == "JONATHAN CASTRO" and nivel == "3BGU":
            continue
        if docente == "JANETH MELO" and nivel == "3BGU":
            continue
        if materia == "CIU_1BGU" and  nivel=="1BGU":
            continue
        if materia == "CIU_2BGU" and  nivel=="2BGU":
            continue

        docentes_en_tiempo[(dia, hora)][docente].add(paralelo)

    for (dia, hora), docentes_dict in docentes_en_tiempo.items():
        for docente, paralelos in docentes_dict.items():
            if len(paralelos) > 1:
                # conflicto = f"[CRUCE] Docente: {docente} asignado a paralelos {sorted(paralelos)} el día {dia}, hora {hora}"
                # print(conflicto)
                penalizaciones += 20 * (len(paralelos) - 1)  # penalización fuerte por cruce real

    return penalizaciones



## 🌿 Evaluación de restricciones blandas en el cromosoma

Esta función calcula penalizaciones asociadas al incumplimiento de **restricciones blandas**, es decir, condiciones deseables que no invalidan el horario pero sí afectan su calidad pedagógica y organizacional.

### 📋 Restricciones consideradas:

1. **🕘 Medio tiempo**
   - Docentes con contrato de medio tiempo no deben impartir clases luego de la hora 10.
   - Penalización: +2 por cada sesión fuera del rango permitido.

2. **👶 Maternidad**
   - Docentes en régimen de maternidad no deben tener sesiones más allá de la hora 14.
   - Penalización: +2 por cada infracción.

3. *(Planificada pero no operativa)*  
   **📚 Diversidad de materias por día**
   - El bloque incluye una estructura para detectar si hay demasiadas materias distintas por día en un aula, pero **actualmente no se registra información en `materias_por_dia`**, por lo tanto **no influye en la penalización** todavía.

### ✨ Propósito:
Estas restricciones permiten incorporar aspectos humanos y pedagógicos que mejoran la **satisfacción del personal docente** y la **fluidez del horario**, influyendo indirectamente en la evolución de soluciones más aceptables.

> 🔧 Esta función se combina en el fitness como `β · penalización_blanda`, con un peso menor que las restricciones duras.


In [ ]:
def evaluar_restricciones_blandas(cromosoma):
    penalizaciones = 0

    # Crear diccionarios rápidos de cargos y docentes
    cargo_docente = dict(zip(docentes["Nombres"], docentes["Cargo"]))

    sesiones_por_dia_docente = defaultdict(lambda: defaultdict(list))  # docente -> día -> horas
    materias_por_dia = defaultdict(set)

    for nivel, aula, dia, hora, materia, docente in cromosoma:
        if not materia.isupper():
            continue

        # 1. Horario de medio tiempo (mañana = hasta 12pm => Hora 8)
        if cargo_docente.get(docente) == "Medio tiempo" and int(hora) > 10:
            penalizaciones += 2

        # 2. Maternidad (última hora debe ser máximo la 14)
        if cargo_docente.get(docente) == "Maternidad" and int(hora) > 14:
            penalizaciones += 2

        # Registrar para posibles evaluaciones futuras
        sesiones_por_dia_docente[docente][dia].append(int(hora))

    for (aula, dia), codigos in materias_por_dia.items():
        if len(codigos) > 3:
            penalizaciones += (len(codigos) - 3)

    return penalizaciones

## 🧮 Función fitness con ponderación de restricciones duras y blandas

Esta función evalúa la calidad (fitness) de un cromosoma considerando las penalizaciones acumuladas por el incumplimiento de restricciones:

### ⚖️ Fórmula utilizada:
$
\text{fitness} = \frac{1}{1 + (\alpha \cdot \text{penalización}_\text{dura} + \beta \cdot \text{penalización}_\text{blanda})}
$

### Parámetros:
- `alpha`: peso asignado a las restricciones duras (por defecto: 1.0).
- `beta`: peso asignado a las restricciones blandas (por defecto: 0.2).

### ¿Qué representa el resultado?
- Un valor cercano a **1.0** indica un cromosoma muy bueno (baja penalización).
- Un valor cercano a **0.0** indica un cromosoma altamente penalizado.

> 🧠 Esta métrica guía el proceso evolutivo, permitiendo seleccionar, cruzar y mutar aquellos cromosomas que respetan más las reglas institucionales del horario escolar.



In [ ]:
# Función fitness con ponderaciones para restricciones duras y blandas
def fitness(cromosoma, alpha=1.0, beta=0.2):
    penal_duras = evaluar_restricciones_duras(cromosoma)
    penal_blandas = evaluar_restricciones_blandas(cromosoma)

    penal_total = alpha * penal_duras + beta * penal_blandas
    score = 1 / (1 + penal_total)

    return score

## 🧩 Función de fitness con evaluación por nivel (split)

Esta función calcula el `fitness` total de un cromosoma y, adicionalmente, descompone el resultado por **nivel educativo** (1EGB, 2EGB, ..., 3BGU). Esto permite realizar análisis más detallados sobre el rendimiento evolutivo de cada subhorario.

### ¿Cómo funciona?

- Se agrupan los genes del cromosoma según su nivel (`gen[0]`).
- Se evalúa cada subconjunto (`subcromosoma`) usando la función tradicional `fitness()`.
- Se guarda el `fitness` por nivel en un diccionario.
- Se retorna tanto el `fitness` total del cromosoma completo como el diccionario por split.

### Retorna:
- `total_fitness`: fitness completo del cromosoma.
- `fitness_por_split`: diccionario con la forma `{nivel: score}`.

### Aplicaciones:
- 📊 Visualización de progreso por nivel en la evolución.
- 📉 Detección de niveles que impiden mejorar el fitness global.
- 🔍 Posibilita estrategias de evolución adaptativa por subgrupo.

> ⚠️ Actualmente, `total_fitness` se recalcula en cada iteración dentro del loop, lo que es redundante. Podría optimizarse llamando a `fitness(cromosoma)` una sola vez antes del bucle.


In [ ]:
def fitness_con_split(cromosoma, alpha=1.0, beta=0.2):
    from collections import defaultdict

    # Agrupar genes por nivel (gen[0] es el nivel)
    genes_por_nivel = defaultdict(list)
    for gen in cromosoma:
        nivel = gen[0]
        genes_por_nivel[nivel].append(gen)

    fitness_por_split = {}
    total_fitness = 0

    for nivel, subcromosoma in genes_por_nivel.items():
        # Calcular fitness del subcromosoma con función tradicional
        score_nivel = fitness(subcromosoma, alpha=alpha, beta=beta)
        fitness_por_split[nivel] = score_nivel
        total_fitness = fitness(cromosoma)

    return total_fitness, fitness_por_split


## 🌱 Generación de la población inicial para el algoritmo genético

Este bloque crea la población base sobre la cual el algoritmo genético comenzará su evolución. Incluye tanto conocimiento previo como diversidad genética.

### Estructura del proceso:

1. **📥 Carga de un cromosoma histórico**
   - Se importa un horario real del colegio (por ejemplo, el del año anterior) desde un archivo `.txt`.
   - Esta solución válida se incluye directamente en la población para preservar una base de calidad.

2. **🧬 Generación de nuevos cromosomas**
   - Se generan cromosomas aleatorios usando la función `generar_cromosoma()`, que respeta las restricciones del problema.
   - Esto garantiza diversidad genética y aumenta las posibilidades de encontrar soluciones óptimas en generaciones futuras.

### Parámetro:
- `tamano_poblacion`: cantidad total de cromosomas iniciales (por defecto: 100).

> Esta combinación de conocimiento previo y soluciones aleatorias acelera la convergencia y mejora la calidad evolutiva desde las primeras generaciones.



In [ ]:
def cargar_cromosoma_desde_txt(nombre_archivo):
    cromosoma = []
    with open(nombre_archivo, "r", encoding="utf-8") as f:
        for linea in f:
            linea = linea.strip()
            if linea:
                cromosoma.append(eval(linea))
    return cromosoma

def generar_poblacion_inicial(tamano_poblacion=100, ruta_horario_valido=ruta + "/Horario_LEV.txt"):
    poblacion = []


    # ✅ 1. Cargar el cromosoma válido del año anterior
    cromosoma_historico = cargar_cromosoma_desde_txt(ruta_horario_valido)
    poblacion.append(cromosoma_historico)

    # ✅ 2. Generar cromosomas adicionales
    while len(poblacion) < tamano_poblacion:
        cromosoma_nuevo = generar_cromosoma()
        poblacion.append(cromosoma_nuevo)

    return poblacion

# Generar la población inicial
poblacion_inicial = generar_poblacion_inicial(tamano_poblacion=100)

🔵 **Flujo general del ciclo genético:**

1. **Evaluar fitness** de la población inicial.
2. **Seleccionar padres** (basado en fitness, por ejemplo, torneo, ruleta, elitismo).
3. **Aplicar crossover** (cruce de padres para generar hijos).
4. **Aplicar mutación** (alterar genes de hijos con cierta probabilidad).
5. **Evaluar fitness** de los nuevos individuos.
6. **Formar nueva población** (reemplazar parcialmente o completamente).
7. **Repetir** desde el paso 2 hasta cumplir el criterio de parada (n generaciones o fitness deseado).


## ⚙️ Evaluación paralela del fitness en la población

Estas funciones permiten evaluar el fitness de múltiples cromosomas en paralelo, acelerando significativamente el proceso evolutivo en cada generación del algoritmo genético.

### Funciones definidas:

- `evaluar2(cromosoma)`: versión directa que llama a `fitness()` sobre un cromosoma.
- `evaluar2_split(cromosoma)`: evalúa un cromosoma completo y retorna el fitness por nivel (`split_info`) mediante `fitness_con_split()`.
- `evaluar_fitness_poblacion(poblacion)`: evalúa toda la población utilizando **hilos paralelos** (`ThreadPoolExecutor`), mejorando el rendimiento computacional.

### Parámetros clave:
- `alpha`, `beta`: ponderaciones para restricciones duras y blandas.
- `num_procesos`: número de núcleos lógicos que se usarán para la evaluación concurrente.

> 💡 La evaluación paralela es crítica cuando el tamaño de la población es grande (e.g., >100 cromosomas), ya que reduce el tiempo por generación sin afectar la calidad de la evaluación.


In [ ]:
from concurrent.futures import ProcessPoolExecutor
import os

def evaluar2_split(cromosoma, alpha=1.0, beta=0.2):
    score, split_info = fitness_con_split(cromosoma, alpha=alpha, beta=beta)
    return score, split_info
def evaluar2(cromosoma, alpha=1.0, beta=0.2):
        return fitness(cromosoma, alpha=alpha, beta=beta)

def evaluar_fitness_poblacion(poblacion, alpha=1.0, beta=0.2, num_procesos=2):
    from concurrent.futures import ThreadPoolExecutor

    with ThreadPoolExecutor(max_workers=num_procesos) as executor:
        resultados = list(executor.map(
            lambda cromo: fitness(cromo, alpha=alpha, beta=beta),
            poblacion
        ))

    return resultados


## 🧠 Evaluación paralela del fitness por nivel (con split) en toda la población

Esta función permite evaluar en paralelo todos los cromosomas de la población, retornando tanto el `fitness global` como un `detalle por nivel educativo` (split) de cada cromosoma.

### Características:
- Usa `ProcessPoolExecutor` para aprovechar múltiples núcleos físicos y evitar el GIL de Python.
- Cada cromosoma es evaluado mediante `evaluar2_split()`, que calcula:
  - `fitness global` total del cromosoma.
  - `fitness por nivel` desglosado.

### ¿Qué retorna?
- `fitness_scores`: lista de valores de fitness global, uno por cromosoma.
- `fitness_por_split`: lista de diccionarios `{nivel: score}`, uno por cromosoma.

### Aplicaciones:
- 🔍 Análisis de convergencia por nivel educativo (1EGB a 3BGU).
- 📊 Visualización de avances diferenciados dentro de la población.
- 🎯 Selección dirigida o estrategias de evolución específicas por subgrupo.

> 🧬 Esta evaluación enriquecida es útil en problemas **multinivel** como el scheduling educativo, donde cada nivel tiene sus propias restricciones, docentes y prioridades.


In [ ]:
def evaluar_fitness_poblacion_con_split(poblacion, alpha=1.0, beta=0.2, num_procesos=None):
    if num_procesos is None:
        num_procesos = os.cpu_count()

    with ProcessPoolExecutor(max_workers=num_procesos) as executor:
        resultados = list(executor.map(evaluar2_split, poblacion))

    # Separar los valores globales y los diccionarios por split
    fitness_scores = [r[0] for r in resultados]
    fitness_por_split = [r[1] for r in resultados]

    return fitness_scores, fitness_por_split


## 🧬 Selección de padres mediante elitismo (fitness general)

Esta función implementa una estrategia de **selección elitista**, donde se escogen los cromosomas con mejor fitness dentro de la población para ser usados como padres en la siguiente generación del algoritmo genético.

### ¿Cómo funciona?
1. Combina cada cromosoma con su respectivo valor de fitness.
2. Ordena la población de mayor a menor score de fitness.
3. Selecciona los `cantidad_padres` mejores individuos (sin duplicación).

### Parámetros:
- `poblacion`: lista de cromosomas.
- `fitness_poblacion`: lista de valores de fitness asociados a la población.
- `cantidad_padres`: número de padres a seleccionar para reproducir.

### Aplicaciones:
- 🧠 Mantiene soluciones de alta calidad a lo largo de las generaciones.
- 🚫 Evita que soluciones válidas se pierdan por azar.
- 📈 Favorece la convergencia rápida del algoritmo.

> 🔍 Este mecanismo se puede complementar con otras estrategias de diversidad como **torneos** o **ruleta**, si se desea evitar estancamientos evolutivos.



In [ ]:
# Función para seleccionar padres usando elitismo con fitness general
def seleccionar_padres_elitismo(poblacion, fitness_poblacion, cantidad_padres=20):
    # Combinar cromosomas con sus valores de fitness
    poblacion_con_fitness = list(zip(poblacion, fitness_poblacion))

    # Ordenar de mayor a menor fitness
    poblacion_con_fitness.sort(key=lambda x: x[1], reverse=True)

    # Seleccionar los mejores 'cantidad_padres' cromosomas
    padres_seleccionados = [individuo for individuo, fitness in poblacion_con_fitness[:cantidad_padres]]

    return padres_seleccionados

# Ejemplo de uso
cantidad_padres = 20
padres = seleccionar_padres_elitismo(poblacion_inicial, fitness_inicial, cantidad_padres=cantidad_padres)

print(f"Total de padres seleccionados: {len(padres)}")

Total de padres seleccionados: 20


## 🧬 Estrategia de Crossover: `crossover_hibrido_heuristico_burke`

Este operador se basa en el enfoque de Burke et al. (1995) adaptado a la estructura y restricciones del problema de generación de horarios escolares con materias sincronizadas, bloques contiguos y restricciones docentes.

---

### 🔢 Fases del Crossover

### 🧩 FASE 1 – Herencia de Bloques Sincronizados
- Se heredan bloques completos por materia y nivel.
- Se selecciona un padre como fuente de la franja horaria (día y hora).
- Se valida disponibilidad de esa franja para todas las aulas del nivel.
- Se insertan todas las asignaciones del bloque, **manteniendo sincronización por nivel**.

**Materias tratadas como bloques sincronizados:**
- `INGLES ESPECIAL` (7EGB–3BGU)
- `CIU_1BGU`, `CIU_2BGU`
- Materias dictadas por `JONATHAN CASTRO` y `JANETH MELO` en 3BGU

---






In [ ]:
from collections import defaultdict

def extraer_bloques_sincronizados_por_materia_y_nivel(cromosoma):
    """
    Extrae bloques sincronizados por (nivel, materia/docente) del cromosoma.
    Devuelve un diccionario: (materia_o_docente, nivel) -> { (día, hora): [genes] }
    """
    bloques = defaultdict(lambda: defaultdict(list))

    for gen in cromosoma:
        nivel, aula, dia, hora, materia, docente = gen

        # INGLÉS ESPECIAL: sincronizado por nivel desde 7EGB en adelante
        if materia == "INGLES ESPECIAL" and nivel in ["7EGB", "8EGB", "9EGB", "10EGB", "1BGU", "2BGU", "3BGU"]:
            bloques[("INGLES ESPECIAL", nivel)][(dia, hora)].append(gen)

        # JONATHAN CASTRO o JANETH MELO en 3BGU
        if (nivel == "3BGU" and materia == "EST_3BGU") or (nivel == "3BGU" and materia == "DES_3BGU"):
            bloques[(materia, nivel)][(dia, hora)].append(gen)

        # Ciudadanía sincronizada
        if (nivel == "1BGU" and materia == "CIU_1BGU") or (nivel == "2BGU" and materia == "CIU_2BGU"):
            bloques[(materia, nivel)][(dia, hora)].append(gen)

    return bloques

from collections import Counter
import pandas as pd

def seleccionar_y_heredar_bloques_sincronizados(padre1, padre2):
    """
    Reescritura basada en el generador:
    - Para cada materia sincronizada por nivel, se identifica en qué franja horaria se asigna en cada padre.
    - Se elige la franja más frecuente y se replica esa franja para todas las aulas del nivel en el hijo.
    """
    hijo = []
    bloques_padres = padre1 + padre2

    materias_especiales = [
        ("INGLES ESPECIAL", "7EGB"), ("INGLES ESPECIAL", "8EGB"), ("INGLES ESPECIAL", "9EGB"),
        ("INGLES ESPECIAL", "10EGB"), ("INGLES ESPECIAL", "1BGU"), ("INGLES ESPECIAL", "2BGU"), ("INGLES ESPECIAL", "3BGU"),
        ("EST_3BGU", "3BGU"), ("DES_3BGU", "3BGU"),
        ("CIU_1BGU", "1BGU"), ("CIU_2BGU", "2BGU")
    ]

    for materia, nivel in materias_especiales:
        # Contar franjas de aparición en los padres
        franjas = [
            (g[2], g[3]) for g in bloques_padres
            if g[4] == materia and g[0] == nivel
        ]
        if not franjas:
            continue

        franja_mas_comun = Counter(franjas).most_common(1)[0][0]
        dia, hora = franja_mas_comun

        # Aplicar a todas las aulas del nivel
        aulas = paralelos.loc[paralelos["Nivel"] == nivel, "Nombre_paralelo"].tolist()
        for aula in aulas:
            docente = obtener_docente_por_materia_codigo(materia)
            hijo.append((nivel, aula, dia, hora, materia, docente))
            # Si es en parejas, añadir la segunda hora
            if materia in ["INGLES ESPECIAL", "EST_3BGU", "DES_3BGU"]:
                hijo.append((nivel, aula, dia, hora + 1, materia, docente))

    return hijo


## 🔗 Extracción de bloques contiguos de materias emparejadas por nivel y aula

Esta función identifica pares de materias que aparecen **de forma contigua** (una seguida de la otra en el horario) y que representan **combinaciones pedagógicas válidas**, como:
- `DER` + `COM`
- `SPE` + `INV`
- `EXP` + `CAL` o `RAZ`
- `DAN` + `MUS` (y viceversa)

### ¿Qué hace?
- Recorre el cromosoma nivel por nivel, aula por aula.
- Busca bloques de dos horas seguidas el mismo día.
- Si detecta un par válido según las reglas pedagógicas, lo registra.


In [ ]:
def extraer_bloques_contiguos_por_nivel(cromosoma):
    """
    Extrae todos los pares de materias contiguas válidas por nivel y aula.
    Devuelve un diccionario: (nivel, aula) -> lista de tuplas [(hora, (materia1, materia2), (dia))]
    """
    pares_validos = {
        "DER": "COM",
        "SPE": "INV",
        "SAL": "ECO",
        "EXP_CAL": ("EXP", "CAL"),
        "EXP_RAZ": ("EXP", "RAZ"),
        "DAN_MUS": ("DAN", "MUS"),
        "MUS_DAN": ("MUS", "DAN")
    }

    bloques_contiguos = defaultdict(list)

    for nivel, aula, dia, hora, materia, _ in cromosoma:
        # Buscar siguiente hora en el mismo día
        siguiente = [
            g for g in cromosoma
            if g[0] == nivel and g[1] == aula and g[2] == dia and g[3] == hora + 1
        ]
        if not siguiente:
            continue

        siguiente_materia = siguiente[0][4]
        par = (materia, siguiente_materia)

        # Evaluar si es un par válido
        if (materia.startswith("DER") and siguiente_materia.startswith("COM")) or \
           (materia.startswith("COM") and siguiente_materia.startswith("DER")) or \
           (materia.startswith("SPE") and siguiente_materia.startswith("INV")) or \
           (materia.startswith("INV") and siguiente_materia.startswith("SPE")) or \
           (materia.startswith("SAL") and siguiente_materia.startswith("ECO")) or \
           (materia.startswith("ECO") and siguiente_materia.startswith("SAL")) or \
           (materia.startswith("EXP") and siguiente_materia.startswith("CAL")) or \
           (materia.startswith("CAL") and siguiente_materia.startswith("EXP")) or \
           (materia.startswith("RAZ") and siguiente_materia.startswith("EXP")) or \
           (materia.startswith("EXP") and siguiente_materia.startswith("RAZ")) or \
           (materia.startswith("DAN") and siguiente_materia.startswith("MUS")) or \
           (materia.startswith("MUS") and siguiente_materia.startswith("DAN")):

            bloques_contiguos[(nivel, aula)].append((dia, hora, (materia, siguiente_materia)))

    return bloques_contiguos


### 🔗 FASE 2 – Inserción de Pares Contiguos (2 bloques)
- Se procesan pares de materias que deben ir juntas y contiguas (ej. `DER–COM`, `SPE–INV`, etc.).
- Se busca su aparición contigua en alguno de los padres.
- Se inserta en el hijo en un bloque de dos periodos consecutivos `(hora, hora+1)`.
- Se respeta la cuota de cada materia y la validez del horario.

---



In [ ]:
# Reimportar librerías necesarias después del reinicio
from collections import defaultdict
import random
import pandas as pd


def seleccionar_y_heredar_materias_en_pares(padre1, padre2, obtener_docente_por_materia_codigo):
    """
    FASE 2 corregida:
    - Hereda pares de materias contiguas desde ambos padres.
    - Solo se asigna un par válido por aula (ej. DER–COM, SPE–INV, etc.).
    - Evita duplicación del mismo par en distintas franjas para un mismo aula.
    """
    hijo = []
    sesiones_hijo = set()

    pares_pref = [
        ("DER", "COM"), ("SPE", "INV"), ("SAL", "ECO"),
        ("EXP", "CAL"), ("EXP", "RAZ"), ("RAZ", "EXP"), ("CAL", "EXP"),
        ("DAN", "MUS"), ("MUS", "DAN")
    ]

    # Combinar pares contiguos válidos desde ambos padres
    bloques_p1 = extraer_bloques_contiguos_por_nivel(padre1)
    bloques_p2 = extraer_bloques_contiguos_por_nivel(padre2)
    claves = set(bloques_p1.keys()).union(bloques_p2.keys())

    asignados_por_aula = defaultdict(set)  # (nivel, aula) -> set de (prefijo1, prefijo2)

    for clave in claves:
        nivel, aula = clave
        bloques1 = bloques_p1.get(clave, [])
        bloques2 = bloques_p2.get(clave, [])
        todos_los_pares = bloques1 + bloques2
        vistos = set()

        for dia, hora, (m1, m2) in todos_los_pares:
            clave_fr = (dia, hora, m1, m2)
            if clave_fr in vistos:
                continue
            vistos.add(clave_fr)

            if (nivel, aula, dia, hora) in sesiones_hijo or (nivel, aula, dia, hora + 1) in sesiones_hijo:
                continue

            # Detectar los prefijos
            pref1, pref2 = m1.split("_")[0], m2.split("_")[0]
            par_actual = tuple(sorted((pref1, pref2)))

            # Validar si es un par permitido
            if par_actual not in [tuple(sorted(p)) for p in pares_pref]:
                continue

            # Verificar si ya fue asignado este par en esa aula
            if par_actual in asignados_por_aula[(nivel, aula)]:
                continue

            # Asignar el par
            d1 = obtener_docente_por_materia_codigo(m1)
            d2 = obtener_docente_por_materia_codigo(m2)
            hijo.append((nivel, aula, dia, hora, m1, d1))
            hijo.append((nivel, aula, dia, hora + 1, m2, d2))

            sesiones_hijo.add((nivel, aula, dia, hora))
            sesiones_hijo.add((nivel, aula, dia, hora + 1))
            asignados_por_aula[(nivel, aula)].add(par_actual)

    return hijo

### ⚙️ FASE 3 – Asignación del Resto de Materias
- Para cada aula, se construye una "bolsa" con materias restantes según cuota.
- Se recorren los slots libres válidos del horario (`es_horario_valido`) e insertan materias restantes de forma aleatoria o por heurística.
- Se asegura que el total de asignaciones por aula sea exactamente **59**.

---


In [ ]:
from collections import defaultdict
import random

from collections import defaultdict
import random

def asignar_materias_restantes_con_herencia(
    cromosoma_actual,
    materias_df,
    paralelos,
    horas_por_materia,
    obtener_docente_por_materia_codigo,
    es_horario_valido,
    padre1,
    padre2
):
    """
    Completa el cromosoma asegurando que cada aula tenga 59 genes.
    Preserva la herencia de horarios desde los padres si es posible.
    Si no, asigna en espacios disponibles como fallback.
    """
    cromosoma = list(cromosoma_actual)
    sesiones_ocupadas = {(n, a, d, h) for n, a, d, h, _, _ in cromosoma}
    cuenta_materias = defaultdict(lambda: defaultdict(int))  # (nivel, aula) -> materia -> count

    # Inicializar conteo por materia por aula
    for n, a, _, _, m, _ in cromosoma:
        cuenta_materias[(n, a)][m] += 1

    niveles = paralelos["Nivel"].unique()
    dias_semana = ["Lunes", "Martes", "Miércoles", "Jueves", "Viernes"]

    for nivel in niveles:
        aulas = paralelos.loc[paralelos["Nivel"] == nivel, "Nombre_paralelo"]
        materias_nivel = materias_df[materias_df["Nivel"] == nivel]["Código"].tolist()

        for aula in aulas:
            genes_aula = [g for g in cromosoma if g[0] == nivel and g[1] == aula]
            total_actual = len(genes_aula)
            if total_actual >= 59:
                continue

            bolsa = []
            for cod in materias_nivel:
                horas_necesarias = horas_por_materia.get(cod, 0)
                actuales = cuenta_materias[(nivel, aula)].get(cod, 0)
                faltantes = max(0, horas_necesarias - actuales)
                bolsa += [cod] * faltantes

            random.shuffle(bolsa)
            for cod in bolsa:
                docente = obtener_docente_por_materia_codigo(cod)
                heredado = False

                # Intentar heredar franja desde padres
                for padre in [padre1, padre2]:
                    for n, a, d, h, m, _ in padre:
                        if n == nivel and a == aula and m == cod and (n, a, d, h) not in sesiones_ocupadas and es_horario_valido(n, d, h):
                            cromosoma.append((n, a, d, h, m, docente))
                            sesiones_ocupadas.add((n, a, d, h))
                            cuenta_materias[(n, a)][m] += 1
                            heredado = True
                            break
                    if heredado:
                        break

                # Si no se pudo heredar, asignar en espacio libre
                if not heredado:
                    for d in dias_semana:
                        for h in range(1, 17):
                            if (nivel, aula, d, h) in sesiones_ocupadas:
                                continue
                            if not es_horario_valido(nivel, d, h):
                                continue
                            cromosoma.append((nivel, aula, d, h, cod, docente))
                            sesiones_ocupadas.add((nivel, aula, d, h))
                            cuenta_materias[(nivel, aula)][cod] += 1
                            break
                        else:
                            continue
                        break

    return cromosoma


### 🛠️ Reglas Generales
- Ningún `(nivel, aula, día, hora)` debe contener más de una asignación.
- No se deben omitir materias, se garantiza un cromosoma completo con **2832 genes**.
- Los docentes se asignan dinámicamente con la función:  
  `obtener_docente_por_materia_codigo(materia)`
- Todo bloque se valida antes de ser insertado para garantizar factibilidad.

---

### 📌 Referencia
Adaptado del paper:  
**"A Hybrid Genetic Algorithm for Highly Constrained Timetabling Problems"**  
Rupert Weare, Edmund Burke & Dave Elliman (1995)

In [ ]:
def crossover_hibrido_heuristico_burke(padre1, padre2):
    """
    Crossover híbrido basado en Burke et al. (1995) adaptado para el problema de scheduling escolar.
    Integra:
    - Fase 1: Herencia de bloques sincronizados por nivel.
    - Fase 2: Herencia de bloques contiguos por aula (pares de materias).
    - Fase 3: Asignación de materias restantes preservando herencia horaria.

    Variables globales requeridas:
    - materias, paralelos, horas_por_materia
    - obtener_docente_por_materia_codigo, es_horario_valido
    """
    # FASE 1: bloques sincronizados (como INGLÉS ESPECIAL o CIU)
    hijo = seleccionar_y_heredar_bloques_sincronizados(padre1, padre2)

    # FASE 2: bloques contiguos válidos (parejas como DER-COM)
    hijo += seleccionar_y_heredar_materias_en_pares(padre1, padre2, obtener_docente_por_materia_codigo)

    # FASE 3: completar con herencia prioritaria
    hijo = asignar_materias_restantes_con_herencia(
        cromosoma_actual=hijo,
        materias_df=materias,
        paralelos=paralelos,
        horas_por_materia=horas_por_materia,
        obtener_docente_por_materia_codigo=obtener_docente_por_materia_codigo,
        es_horario_valido=es_horario_valido,
        padre1=padre1,
        padre2=padre2
    )

    return ordenar_cromosoma(hijo)

# Ejemplo de uso
hijo_ejemplo = crossover_hibrido_heuristico_burke(padres[3], padres[5]) #2 y  10

# Inspección de genes
print(f"Total de genes generados: {len(hijo_ejemplo)}")
for gen in hijo_ejemplo[:]:
    print(gen)


Total de genes generados: 2832
('1EGB', 'Ejemplares', 'Lunes', 1, 'MAT_1EGB', 'PAULINA ÑACATO')
('1EGB', 'Ejemplares', 'Lunes', 2, 'PEN_1EGB', 'FERNANDA CAÑIZARES')
('1EGB', 'Ejemplares', 'Lunes', 3, 'MAT_1EGB', 'PAULINA ÑACATO')
('1EGB', 'Ejemplares', 'Lunes', 4, 'LEN_1EGB', 'MALENA OÑA')
('1EGB', 'Ejemplares', 'Lunes', 6, 'EXT_1EGB', 'MARIEL RODRIGUEZ')
('1EGB', 'Ejemplares', 'Lunes', 7, 'LEC_1EGB', 'ALISON PLAZA')
('1EGB', 'Ejemplares', 'Lunes', 8, 'MAT_1EGB', 'PAULINA ÑACATO')
('1EGB', 'Ejemplares', 'Lunes', 9, 'ING_1EGB', 'MARCIA HERRERA VÁSQUEZ')
('1EGB', 'Ejemplares', 'Lunes', 10, 'MAT_1EGB', 'PAULINA ÑACATO')
('1EGB', 'Ejemplares', 'Lunes', 13, 'MAT_1EGB', 'PAULINA ÑACATO')
('1EGB', 'Ejemplares', 'Lunes', 14, 'PEN_1EGB', 'FERNANDA CAÑIZARES')
('1EGB', 'Ejemplares', 'Martes', 1, 'ECO_1EGB', 'MARJURIE JÁCOME')
('1EGB', 'Ejemplares', 'Martes', 2, 'SAL_1EGB', 'VANESSA ESPÍN')
('1EGB', 'Ejemplares', 'Martes', 3, 'MAT_1EGB', 'PAULINA ÑACATO')
('1EGB', 'Ejemplares', 'Martes', 4, 'MAT_

In [ ]:
# Uso:
ctr = contar_asignaciones_por_aula(hijo_ejemplo)
for (nivel, aula), num_genes in ctr.items():
    print(f"{nivel} • {aula}: {num_genes}")


1EGB • Ejemplares: 59
1EGB • Entusiastas: 59
1EGB • Estudiosos: 59
1EGB • Exitosos: 59
2EGB • Afectuosos: 59
2EGB • Amables: 59
2EGB • Amigables: 59
2EGB • Amistosos: 59
3EGB • Colaboradores: 59
3EGB • Conciliadores: 59
3EGB • Cordiales: 59
3EGB • Creativos: 59
4EGB • Optimistas: 59
4EGB • Ordenados: 59
4EGB • Organizados: 59
4EGB • Originales: 59
5EGB • Admirables: 59
5EGB • Afectivos: 59
5EGB • Aplicados: 59
5EGB • Atentos: 59
6EGB • Ecuánimes: 59
6EGB • Emprendedores: 59
6EGB • Excelentes: 59
6EGB • Expresivos: 59
7EGB • Reflexivos: 59
7EGB • Resilientes: 59
7EGB • Respetuosos: 59
7EGB • Responsables: 59
8EGB • Decididos: 59
8EGB • Diligentes: 59
8EGB • Dinámicos: 59
8EGB • Disciplinados: 59
9EGB • Altruistas: 59
9EGB • Asertivos: 59
9EGB • Audaces: 59
9EGB • Auténticos: 59
10EGB • Pacifistas: 59
10EGB • Perspicaces: 59
10EGB • Positivos: 59
1BGU • Confiables: 59
1BGU • Integros: 59
1BGU • Leales: 59
2BGU • Honestos: 59
2BGU • Justos: 59
2BGU • Proactivos: 59
3BGU • Empáticos: 59
3B

In [ ]:
# Ejemplo de uso:
df_horas = contar_horas_por_materia(hijo_ejemplo)
df_horas

,Nivel,Aula,Código_materia,Horas_asignadas
0,10EGB,Pacifistas,BIO_10EGB,3
1,10EGB,Pacifistas,COM_10EGB,1
2,10EGB,Pacifistas,DAN_10EGB,1
3,10EGB,Pacifistas,DER_10EGB,1
4,10EGB,Pacifistas,DES_10EGB,2
...,...,...,...,...
1095,9EGB,Auténticos,QUI_9EGB,6
1096,9EGB,Auténticos,ROB_9EGB,2
1097,9EGB,Auténticos,SAL_9EGB,1
1098,9EGB,Auténticos,SOC_9EGB,3


In [ ]:
faltantes_df = verificar_materias_faltantes(hijo_ejemplo, materias, paralelos)
display(faltantes_df)


,Nivel,Aula,Materias,Código,Horas_requeridas,Horas_asignadas,Faltan


## 🧬 Comparación de herencia genética entre padres e hijo

Esta función permite visualizar de manera detallada **de dónde proviene cada gen** (asignación de materia en una franja horaria) dentro de un cromosoma hijo, en relación con sus dos padres.

### ¿Qué evalúa?
Para cada `(nivel, aula, día, hora)`:
- Si la materia del hijo coincide con la del Padre 1 → 🟦
- Si coincide con la del Padre 2 → 🟩
- Si no coincide con ninguno (nuevo valor) → 🟧 (Reconstruido)

### Resultado:
Imprime un reporte por aula, donde cada línea muestra:
- El día
- La hora
- La materia asignada
- La fuente del gen (Padre 1, Padre 2 o Reconstruido)

### Aplicaciones:
- 🔍 Auditoría del operador de crossover.
- 📊 Evaluación cualitativa de la diversidad genética del hijo.
- 🧠 Análisis de cuánta herencia se preserva y cuánta se genera de forma aleatoria o heurística.

> Esta herramienta es útil tanto en etapas de debugging como para justificar la implementación de estrategias de herencia controlada en el algoritmo evolutivo.


In [ ]:
from collections import defaultdict

def comparar_herencia_genes(padre1, padre2, hijo):
    print("🔍 Comparación de herencia gen por gen\n")

    genes_p1 = {(n, a, d, h): m for n, a, d, h, m, _ in padre1}
    genes_p2 = {(n, a, d, h): m for n, a, d, h, m, _ in padre2}

    herencia_por_bloque = defaultdict(list)

    for gen in hijo:
        nivel, aula, dia, hora, materia_h, _ = gen
        key = (nivel, aula, dia, hora)

        materia_p1 = genes_p1.get(key)
        materia_p2 = genes_p2.get(key)

        if materia_h == materia_p1:
            fuente = "🟦 Padre 1"
        elif materia_h == materia_p2:
            fuente = "🟩 Padre 2"
        else:
            fuente = "🟧 Reconstruido"

        herencia_por_bloque[(nivel, aula)].append((key, materia_h, fuente))

    # Mostrar agrupado
    for (nivel, aula), registros in sorted(herencia_por_bloque.items()):
        print(f"\n📚 Nivel: {nivel} | Aula: {aula}")
        print("-" * 60)
        for (n, a, d, h), mat, src in sorted(registros, key=lambda x: (x[0][2], x[0][3])):  # orden por día y hora
            print(f"{d:<10} {h:>2}h → {mat:<12} ← {src}")
        print("-" * 60)


In [ ]:
comparar_herencia_genes(padres[3], padres[5], hijo_ejemplo)


🔍 Comparación de herencia gen por gen


📚 Nivel: 10EGB | Aula: Pacifistas
------------------------------------------------------------
Jueves      1h → SOC_10EGB    ← 🟦 Padre 1
Jueves      2h → DES_10EGB    ← 🟦 Padre 1
Jueves      3h → BIO_10EGB    ← 🟦 Padre 1
Jueves      4h → PLA_10EGB    ← 🟦 Padre 1
Jueves      5h → PEN_10EGB    ← 🟦 Padre 1
Jueves      7h → QUI_10EGB    ← 🟦 Padre 1
Jueves      8h → EMP_10EGB    ← 🟦 Padre 1
Jueves      9h → LEN_10EGB    ← 🟦 Padre 1
Jueves     10h → LEC_10EGB    ← 🟦 Padre 1
Jueves     11h → FIS_10EGB    ← 🟦 Padre 1
Jueves     12h → GEO_10EGB    ← 🟦 Padre 1
Jueves     15h → PRO_10EGB    ← 🟦 Padre 1
Jueves     16h → QUI_10EGB    ← 🟦 Padre 1
Lunes       1h → PEN_10EGB    ← 🟦 Padre 1
Lunes       2h → LEN_10EGB    ← 🟦 Padre 1
Lunes       3h → ECO_10EGB    ← 🟦 Padre 1
Lunes       4h → SAL_10EGB    ← 🟦 Padre 1
Lunes       5h → FIS_10EGB    ← 🟦 Padre 1
Lunes       7h → GEO_10EGB    ← 🟦 Padre 1
Lunes       8h → MAT_10EGB    ← 🟦 Padre 1
Lunes       9h → MAT_10EG

## 🔁 Operador de mutación: intercambio de materias dentro del mismo nivel

Este operador de mutación introduce variación genética en el cromosoma (hijo) al intercambiar aleatoriamente dos asignaciones (genes) **dentro del mismo nivel educativo**, preservando así la coherencia estructural del horario.

### 🧬 ¿Cómo funciona?

1. Para cada nivel (e.g. 6EGB, 1BGU), se identifica la sublista de genes correspondientes.
2. Con una probabilidad `prob_mutacion`, se seleccionan dos posiciones aleatorias dentro de ese nivel.
3. Se intercambian las **materias** y los **docentes correspondientes** entre esas dos posiciones.
4. Las demás propiedades del gen (nivel, aula, día, hora) se conservan.

### Parámetro:
- `prob_mutacion`: probabilidad con la que se aplica el intercambio en cada nivel (por defecto: 10%).

### Ventajas:
- 🔄 Mantiene la asignación horaria constante (no cambia días ni horas).
- 🎲 Introduce variabilidad sin romper la estructura del horario.
- 📊 Ayuda a escapar de óptimos locales durante la evolución.

> 💡 Este tipo de mutación es especialmente útil cuando el crossover ya genera soluciones válidas pero se desea **refinar o explorar nuevas combinaciones** sin perturbar demasiado la factibilidad.



In [ ]:
import random
from copy import deepcopy
# Función de mutación: intercambio de materias en el mismo nivel
def mutacion_intercambio(hijo, prob_mutacion=0.1):
    hijo_mutado = deepcopy(hijo)

    niveles = set(gen[0] for gen in hijo_mutado)

    for nivel in niveles:
        # Obtener índices del nivel actual
        indices_nivel = [i for i, gen in enumerate(hijo_mutado) if gen[0] == nivel]

        if len(indices_nivel) >= 2:
            # Con probabilidad prob_mutacion, aplicar swap
            if random.random() < prob_mutacion:
                idx1, idx2 = random.sample(indices_nivel, 2)

                # Intercambiar materias
                materia1 = hijo_mutado[idx1][4]
                materia2 = hijo_mutado[idx2][4]

                # Reasignar materias y docentes intercambiados
                nivel1, aula1, dia1, hora1, _, _ = hijo_mutado[idx1]
                nivel2, aula2, dia2, hora2, _, _ = hijo_mutado[idx2]

                docente1 = obtener_docente_por_materia_codigo(materia1)
                docente2 = obtener_docente_por_materia_codigo(materia2)

                hijo_mutado[idx1] = (nivel1, aula1, dia1, hora1, materia2, docente2)
                hijo_mutado[idx2] = (nivel2, aula2, dia2, hora2, materia1, docente1)

    return hijo_mutado

# Ejemplo de uso:
hijo_mutado = mutacion_intercambio(hijo_ejemplo, prob_mutacion=0.1)

# Inspección visual
print("Primeros 10 genes del hijo después de la mutación:")
for gen in hijo_mutado[:10]:
    print(gen)


Primeros 10 genes del hijo después de la mutación:
('1EGB', 'Ejemplares', 'Lunes', 1, 'MAT_1EGB', 'PAULINA ÑACATO')
('1EGB', 'Ejemplares', 'Lunes', 2, 'PEN_1EGB', 'FERNANDA CAÑIZARES')
('1EGB', 'Ejemplares', 'Lunes', 3, 'MAT_1EGB', 'PAULINA ÑACATO')
('1EGB', 'Ejemplares', 'Lunes', 4, 'LEN_1EGB', 'MALENA OÑA')
('1EGB', 'Ejemplares', 'Lunes', 6, 'EXT_1EGB', 'MARIEL RODRIGUEZ')
('1EGB', 'Ejemplares', 'Lunes', 7, 'LEC_1EGB', 'ALISON PLAZA')
('1EGB', 'Ejemplares', 'Lunes', 8, 'MAT_1EGB', 'PAULINA ÑACATO')
('1EGB', 'Ejemplares', 'Lunes', 9, 'ING_1EGB', 'MARCIA HERRERA VÁSQUEZ')
('1EGB', 'Ejemplares', 'Lunes', 10, 'MAT_1EGB', 'PAULINA ÑACATO')
('1EGB', 'Ejemplares', 'Lunes', 13, 'MAT_1EGB', 'PAULINA ÑACATO')


## 🧬 Generación de nueva población a partir de padres seleccionados (elitismo + crossover + mutación)

Esta función implementa el ciclo reproductivo del algoritmo genético: a partir de la élite seleccionada mediante fitness, se genera una nueva población mediante **cruce híbrido heurístico** y **mutación estructurada**.

### 🧱 ¿Cómo funciona?

1. Se toman pares aleatorios de la lista de `padres`.
2. Se aplica el operador `crossover_hibrido_heuristico_burke()` para producir un hijo.
3. Se aplica `mutacion_intercambio()` sobre el hijo resultante.
4. El nuevo cromosoma es agregado a la nueva población.

### Parámetro:
- `prob_mutacion`: probabilidad de aplicar mutación a cada hijo generado.

### Resultado:
Devuelve una lista con la **nueva generación** de cromosomas, del mismo tamaño que la cantidad de padres.

### Aplicaciones:
- 🔄 Introduce recombinación genética para explorar el espacio de soluciones.
- 🌱 Preserva parcialmente el conocimiento de los padres mientras se generan soluciones nuevas.
- 📉 Junto con elitismo, permite conservar la calidad mientras se mantiene diversidad.

> 🔍 El crossover implementado sigue un enfoque híbrido basado en Burke et al. (2004), que favorece la herencia de bloques sincronizados y pares pedagógicos clave.



In [ ]:
# Función para generar una nueva población a partir de la élite
def generar_nueva_generacion(padres, prob_mutacion=0.1):
    nueva_generacion = []

    # Asumiendo que padres ya es una lista de individuos seleccionados
    cantidad_padres = len(padres)

    for _ in range(cantidad_padres):  # Generar tantos hijos como padres disponibles
        # Seleccionar dos padres aleatoriamente
        padre1, padre2 = random.sample(padres, 2)

        # Crossover
        hijo = crossover_hibrido_heuristico_burke(padre1, padre2)

        # Mutación
        hijo_mutado = mutacion_intercambio(hijo, prob_mutacion=prob_mutacion)

        # Agregar hijo mutado a la nueva generación
        nueva_generacion.append(hijo_mutado)

    return nueva_generacion


# Ejemplo de uso:
# padres = lista de cromosomas seleccionados con elitismo

nueva_generacion = generar_nueva_generacion(padres, prob_mutacion=0.1)

# Mostrar el tamaño de la nueva generación
print(f"Número de hijos generados: {len(nueva_generacion)}")


Número de hijos generados: 20


## 🧬 Formación de nueva población combinando élite y descendencia (herencia controlada)

Esta función consolida una nueva población genética seleccionando los mejores individuos entre los padres (élite) y los hijos generados, garantizando así la **herencia de calidad** y la **diversidad genética** en cada generación.

### 🧱 ¿Qué hace?

1. Evalúa el `fitness` de todos los cromosomas: padres + hijos.
2. Selecciona los `tamano_poblacion_final` mejores individuos (elitismo global).
3. Si faltan individuos, genera nuevos hijos usando crossover + mutación.
4. Asegura que la población resultante tenga el tamaño correcto.

### Parámetros:
- `padres`: lista de cromosomas seleccionados por elitismo.
- `hijos`: lista de nuevos cromosomas generados.
- `tamano_poblacion_final`: tamaño deseado para la siguiente generación.
- `prob_mutacion`: probabilidad de aplicar mutación a los hijos nuevos.

### Ventajas:
- 🧠 Se preservan soluciones de alta calidad generadas previamente.
- 🔄 Se promueve la exploración del espacio de búsqueda con nuevos cromosomas.
- 📉 Reduce el riesgo de estancamiento evolutivo manteniendo diversidad genética.

> ⚠️ Esta función **revalúa todo el conjunto** antes de decidir qué individuos conservar, lo que permite ajustes dinámicos por generación.



In [ ]:
# Función para formar la nueva población respetando la herencia genética
def formar_nueva_poblacion_heredada(padres, hijos, tamano_poblacion_final=100, prob_mutacion=0.1):
    # Combinar padres élite y nuevos hijos
    poblacion_completa = padres + hijos

    # Evaluar fitness de la población completa
    fitness_completo = evaluar_fitness_poblacion(poblacion_completa)
    poblacion_fitness = list(zip(poblacion_completa, fitness_completo))

    # Ordenar por fitness de mayor a menor
    poblacion_fitness.sort(key=lambda x: x[1], reverse=True)

    # Seleccionar los mejores cromosomas disponibles
    nueva_poblacion = [individuo for individuo, _ in poblacion_fitness[:tamano_poblacion_final]]

    # Verificar si faltan individuos
    faltantes = tamano_poblacion_final - len(nueva_poblacion)

    # Completar los faltantes con hijos generados mediante crossover y mutación
    if faltantes > 0:
        print(f"🔄 Generando {faltantes} individuos adicionales con crossover entre padres para completar la población.")
        while len(nueva_poblacion) < tamano_poblacion_final:
            padre1, padre2 = random.sample(padres, 2)
            hijo = crossover_hibrido_heuristico_burke(padre1, padre2)
            hijo_mutado = mutacion_intercambio(hijo, prob_mutacion=prob_mutacion)
            nueva_poblacion.append(hijo_mutado)

    # Truncar si por alguna razón se excedió el tamaño
    return nueva_poblacion[:tamano_poblacion_final]

# Ejemplo de uso
nueva_poblacion = formar_nueva_poblacion_heredada(padres, nueva_generacion, tamano_poblacion_final=100, prob_mutacion=0.1)

# Mostrar el tamaño final
print(f"✅ Número total de individuos en la nueva población: {len(nueva_poblacion)}")

🔄 Generando 60 individuos adicionales con crossover entre padres para completar la población.
✅ Número total de individuos en la nueva población: 100


## 📊 Análisis de fitness promedio por nivel educativo (split)

Esta función imprime el **fitness promedio por nivel** (e.g. 1EGB, 2EGB, ..., 3BGU) calculado sobre toda la población. Se utiliza la función `fitness_con_split()`, que descompone el fitness global en subcomponentes por nivel educativo.

### ¿Qué hace?

1. Recorre cada cromosoma en la población.
2. Calcula el `fitness` por nivel para ese individuo.
3. Acumula los scores por nivel y calcula su promedio final.
4. Imprime el resultado ordenado.

### Aplicaciones:
- 🧠 Permite identificar **niveles problemáticos** que bajan el fitness global.
- 📉 Apoya decisiones para **focalizar mejoras evolutivas**.
- 📈 Útil para análisis exploratorio o como criterio para detención anticipada por subgrupo.

> 💡 Complementa perfectamente las gráficas de evolución general, añadiendo un análisis interno por subconjuntos funcionales del problema.


In [ ]:
def imprimir_fitness_por_split(poblacion, alpha=1.0, beta=0.2):
    from collections import defaultdict
    acumulado = defaultdict(float)
    for cromosoma in poblacion:
        _, split_scores = fitness_con_split(cromosoma, alpha, beta)
        for split, valor in split_scores.items():
            acumulado[split] += valor

    print("\n📊 Fitness promedio por nivel:")
    for split, total in acumulado.items():
        promedio = total / len(poblacion)
        print(f"  ➤ {split}: {promedio:.2f}")


## 🔁 Ciclo principal de evolución genética multigeneracional

Esta función orquesta el proceso completo de evolución genética para la optimización de horarios escolares, iterando sobre múltiples generaciones con evaluación, selección, cruce, mutación y reemplazo.

### 🧠 ¿Qué hace esta función?

1. Inicializa la población con cromosomas candidatos.
2. Itera hasta un máximo de `n_generaciones`:
   - Evalúa fitness total y por nivel (split).
   - Registra la historia del mejor y promedio fitness.
   - Imprime e informa el progreso en consola y en archivos de log.
   - Aplica selección por elitismo, crossover y mutación.
   - Forma una nueva población respetando el tamaño requerido.
3. Devuelve la última población y todos los historiales útiles.

### Parámetros:
- `poblacion_inicial`: lista de cromosomas iniciales.
- `evaluar_fitness_poblacion`: función para calcular fitness general.
- `evaluar_fitness_poblacion_con_split`: evaluación con desglose por nivel.
- `seleccionar_padres_elitismo`: función de selección de élite.
- `generar_nueva_generacion`: función de crossover + mutación.
- `formar_nueva_poblacion_heredada`: combinación de élite e hijos.
- `n_generaciones`: número máximo de generaciones (default: 1000).
- `tamano_poblacion`: tamaño de cada generación.
- `cantidad_padres`: número de padres para reproducción.
- `prob_mutacion`: probabilidad de mutación aplicada a los hijos.
- `ruta_log`, `nombre_log`: para guardar trazabilidad de la evolución.

### Devuelve:
- `poblacion`: población final.
- `historial_mejor_fitness`: evolución del mejor fitness.
- `historial_fitness_promedio`: evolución del fitness promedio.
- `fitness_poblacion`: valores de fitness de la última generación.
- `historial_split_por_generacion`: historial de fitness por nivel educativo.

### Aplicaciones clave:
- 📈 Seguimiento detallado del desempeño por generación.
- 🧬 Evolución progresiva con conservación de soluciones de calidad.
- 🧪 Control fino sobre cada etapa del proceso evolutivo.

> Esta función constituye el **núcleo operativo** del algoritmo genético aplicado al Educational Timetabling Problem (ECTP).


In [ ]:
from datetime import datetime
from collections import defaultdict
import os

def evolucionar_generaciones_multinivel(
    poblacion_inicial,
    evaluar_fitness_poblacion,
    evaluar_fitness_poblacion_con_split,
    seleccionar_padres_elitismo,
    generar_nueva_generacion,
    formar_nueva_poblacion_heredada,
    n_generaciones=1000,
    tamano_poblacion=1000,
    cantidad_padres=200,
    prob_mutacion=0.1,
    ruta_log=None,
    nombre_log=None
):
    poblacion = poblacion_inicial
    historial_mejor_fitness = []
    historial_fitness_promedio = []
    historial_split_por_generacion = []

    if ruta_log:
        with open(ruta_log, "w", encoding="utf-8") as f:
            f.write(f"📈 Evolución iniciada: {datetime.now()} {nombre_log or ''}\n\n")

    for generacion in range(n_generaciones):
        print(f"\n🚀 Generación {generacion + 1}/{n_generaciones}")

        fitness_poblacion = evaluar_fitness_poblacion(
            poblacion, alpha=1.0, beta=0.2, num_procesos=os.cpu_count()
        )

        # Evaluación por split cada 50 generaciones
        if generacion % 50 == 0:
            _, splits_poblacion = evaluar_fitness_poblacion_con_split(
                poblacion, alpha=1.0, beta=0.2, num_procesos=os.cpu_count()
            )
            acumulado = defaultdict(float)
            cuenta = defaultdict(int)
            for split_dict in splits_poblacion:
                for split, valor in split_dict.items():
                    acumulado[split] += valor
                    cuenta[split] += 1
            promedio_por_split = {
                split: acumulado[split] / cuenta[split]
                for split in acumulado
            }
            historial_split_por_generacion.append(promedio_por_split)

            print(f"\n📊 Resumen fitness por nivel:")
            for nivel, score in promedio_por_split.items():
                print(f"  ➤ {nivel}: {score:.4f}")

            if ruta_log:
                with open(ruta_log, "a", encoding="utf-8") as f:
                    f.write(f"\n📊 Nivel por split - Gen {generacion+1} {nombre_log or ''}:\n")
                    for nivel, score in promedio_por_split.items():
                        f.write(f"  ➤ {nivel}: {score:.4f}\n")

        # Estadísticas principales
        mejor_fitness = max(fitness_poblacion)
        fitness_promedio = sum(fitness_poblacion) / len(fitness_poblacion)
        historial_mejor_fitness.append(mejor_fitness)
        historial_fitness_promedio.append(fitness_promedio)

        print(f"⭐ Mejor fitness: {mejor_fitness:.6f}")
        print(f"📊 Fitness promedio: {fitness_promedio:.6f}")

        if ruta_log:
            with open(ruta_log, "a", encoding="utf-8") as f:
                f.write(f"Gen {generacion+1}: mejor = {mejor_fitness:.5f}, promedio = {fitness_promedio:.5f}\n")

        if ruta_log and (generacion + 1) % 50 == 0:
            top10 = sorted(zip(poblacion, fitness_poblacion), key=lambda x: x[1], reverse=True)[:10]
            with open(ruta_log, "a", encoding="utf-8") as f:
                f.write(f"\n🔟 Top 10 {nombre_log or ''} - Gen {generacion+1}:\n")
                for i, (cromo, fit) in enumerate(top10):
                    f.write(f"#{i+1}: fitness = {fit:.5f} - genes = {len(cromo)}\n")
                    for gen in cromo[:5]:
                        f.write(f"   {gen}\n")
                f.write("\n")

        padres = seleccionar_padres_elitismo(poblacion, fitness_poblacion, cantidad_padres=cantidad_padres)
        hijos = generar_nueva_generacion(padres, prob_mutacion=prob_mutacion)
        poblacion = formar_nueva_poblacion_heredada(
            padres, hijos, tamano_poblacion_final=tamano_poblacion, prob_mutacion=prob_mutacion
        )

    return poblacion, historial_mejor_fitness, historial_fitness_promedio, fitness_poblacion, historial_split_por_generacion

## 🏆 Guardado de los 3 mejores cromosomas (Top 3 global)

Esta función selecciona los tres mejores cromosomas de la población final con base en su fitness y los guarda en archivos `.txt` individuales.

### ¿Qué guarda?
- La posición del cromosoma en el ranking (`#1`, `#2`, `#3`).
- Su puntaje de fitness.
- La secuencia completa de genes (asignaciones horarias).

### Parámetros:
- `poblacion_final`: lista de cromosomas resultantes de la evolución.
- `fitness_final`: lista de valores de fitness asociados.
- `ruta_salida_base`: carpeta donde se guardarán los archivos.

> 💾 Ideal para conservar las mejores soluciones y analizarlas posteriormente.


In [ ]:
import os

def guardar_top3_cromosomas(poblacion_final, fitness_final, ruta_salida_base):
    top3 = sorted(zip(poblacion_final, fitness_final), key=lambda x: x[1], reverse=True)[:3]
    for i, (cromo, fit) in enumerate(top3, start=1):
        ruta_cromo = os.path.join(ruta_salida_base, f"mejor_GLOBAL_#{i}.txt")
        with open(ruta_cromo, "w", encoding="utf-8") as f:
            f.write(f"# Cromosoma top #{i} GLOBAL - fitness: {fit:.5f}\n")
            for gen in cromo:
                f.write(str(gen) + "\n")

## 📈 Registro histórico de la evolución del fitness

Esta función almacena en un archivo `.csv` el comportamiento del fitness a lo largo de las generaciones. Incluye:
- `Fitness_Max`: mejor score de cada generación.
- `Fitness_Prom`: score promedio de la población por generación.
- `Fitness_Min`: el menor valor entre el mejor y promedio (proxy conservador).

### Parámetros:
- `hist_mejor`: lista con el mejor fitness por generación.
- `hist_prom`: lista con el fitness promedio por generación.
- `ruta_csv`: ruta del archivo donde se guarda el historial.

### Aplicaciones:
- 📊 Análisis de convergencia del algoritmo.
- 🔬 Comparación entre configuraciones evolutivas.
- 📉 Identificación de estancamiento o mejora progresiva.

> 💡 Este archivo es esencial para construir gráficos de evolución y evaluar la efectividad del algoritmo.


In [ ]:
import pandas as pd

def guardar_evolucion_fitness(hist_mejor, hist_prom, ruta_csv):
    hist_min = [min(m, p) for m, p in zip(hist_mejor, hist_prom)]
    df = pd.DataFrame({
        "Generacion": list(range(1, len(hist_mejor) + 1)),
        "Fitness_Max": hist_mejor,
        "Fitness_Prom": hist_prom,
        "Fitness_Min": hist_min
    })
    df.to_csv(ruta_csv, index=False, encoding="utf-8")
    return df

## 🧪 Ejecución principal del algoritmo genético y guardado de resultados

Este bloque ejecuta la función principal de evolución `evolucionar_generaciones_multinivel` y almacena los resultados obtenidos para su posterior análisis.

### ¿Qué hace?

1. **Inicializa los parámetros clave**:
   - Ruta de log para seguimiento de la evolución.
   - Nombre descriptivo del experimento (e.g., `EVOLUCIÓN COMPLETA MULTINIVEL`).

2. **Ejecuta la evolución** de la población por 1000 generaciones:
   - Evalúa fitness general y por nivel.
   - Aplica selección, crossover y mutación.
   - Registra el historial de evolución.

3. **Guarda los resultados**:
   - `Top 3` de los mejores cromosomas en archivos `.txt`.
   - Evolución del fitness (`máximo`, `promedio`, `mínimo`) en un archivo `.csv`.

### Salidas esperadas:
- `poblacion_final`: cromosomas resultantes tras la evolución.
- `hist_mejor`: mejor fitness por generación.
- `hist_prom`: fitness promedio por generación.
- `fitness_final`: fitness actual de la última generación.
- `split_hist`: evolución de fitness por nivel.

> 📁 Los resultados son guardados automáticamente en la carpeta `/Tesis_MSDS/logs_evolucion` para su trazabilidad y análisis posterior.


In [ ]:
# ✅ Definir ruta para guardar el log
ruta_log = "/content/drive/MyDrive/Tesis_MSDS/logs_evolucion/evolucion_COMPLETA.txt"
nombre_log = "[EVOLUCIÓN COMPLETA MULTINIVEL]"

# ✅ Llamar a la función
poblacion_final, hist_mejor, hist_prom, fitness_final, split_hist = evolucionar_generaciones_multinivel(
    poblacion_inicial=poblacion_inicial,
    evaluar_fitness_poblacion=evaluar_fitness_poblacion,
    evaluar_fitness_poblacion_con_split=evaluar_fitness_poblacion_con_split,
    seleccionar_padres_elitismo=seleccionar_padres_elitismo,
    generar_nueva_generacion=generar_nueva_generacion,
    formar_nueva_poblacion_heredada=formar_nueva_poblacion_heredada,
    n_generaciones=1000,
    tamano_poblacion=100,
    cantidad_padres=20,
    prob_mutacion=0.1,
    ruta_log=ruta_log,
    nombre_log=nombre_log
)
# ✅ Guardar resultados e histórico
guardar_top3_cromosomas(poblacion_final, fitness_final, "/content/drive/MyDrive/Tesis_MSDS/logs_evolucion")
ruta_csv_fitness = "/content/drive/MyDrive/Tesis_MSDS/logs_evolucion/evolucion_fitness.csv"
df_fitness = guardar_evolucion_fitness(hist_mejor, hist_prom, ruta_csv_fitness)

Streaming output truncated to the last 5000 lines.

🚀 Generación 55/1000
⭐ Mejor fitness: 0.833333
📊 Fitness promedio: 0.016662
🔄 Generando 60 individuos adicionales con crossover entre padres para completar la población.

🚀 Generación 56/1000
⭐ Mejor fitness: 0.833333
📊 Fitness promedio: 0.016486
🔄 Generando 60 individuos adicionales con crossover entre padres para completar la población.

🚀 Generación 57/1000
⭐ Mejor fitness: 0.833333
📊 Fitness promedio: 0.016471
🔄 Generando 60 individuos adicionales con crossover entre padres para completar la población.

🚀 Generación 58/1000
⭐ Mejor fitness: 0.833333
📊 Fitness promedio: 0.016407
🔄 Generando 60 individuos adicionales con crossover entre padres para completar la población.

🚀 Generación 59/1000
⭐ Mejor fitness: 0.833333
📊 Fitness promedio: 0.016492
🔄 Generando 60 individuos adicionales con crossover entre padres para completar la población.

🚀 Generación 60/1000
⭐ Mejor fitness: 0.833333
📊 Fitness promedio: 0.016182
🔄 Generando 60 in


# 🧬 Pipeline del Algoritmo Genético para Generación de Horarios Escolares

Este pipeline resume la implementación completa del algoritmo genético personalizado para resolver el **Educational Course Timetabling Problem (ECTP)** en el contexto del Colegio Lev Vygotsky. Se detalla el flujo modular del código en etapas funcionales.

---

## 1. 🎯 Montaje de entorno y carga de datos
- Montaje de Google Drive.
- Lectura de archivos CSV: docentes, materias, paralelos, horarios.

## 2. 🧼 Preprocesamiento de datos
- Unificación de docentes (e.g., inglés y educación física).
- Normalización de códigos y separación por materia.
- Generación del diccionario `dic_profesores`.

## 3. 🧠 Reglas de negocio y validación horaria
- Definición de restricciones duras por recesos, almuerzos, clubs, evaluaciones.
- Validación de horarios disponibles (`es_horario_valido`).

## 4. 🧱 Generación de cromosomas
- Asignación de bloques sincronizados (e.g., INGLÉS, EST, DES).
- Inserción de pares de materias preferidas (DER–COM, SPE–INV, etc.).
- Relleno final con asignaciones aleatorias válidas.

## 5. 🧬 Evaluación de fitness
- Penalizaciones por:
  - Discontinuidad.
  - Más de 3 horas seguidas.
  - Cruces docentes.
  - Restricciones blandas (medio tiempo, maternidad).
- Fitness global y desagregado por nivel (`fitness_con_split`).

## 6. 🌱 Evolución poblacional
- Generación de población inicial (opcionalmente desde cromosoma histórico).
- Selección de élite.
- Crossover híbrido heurístico.
- Mutación por intercambio dentro de nivel.
- Formación de nueva población respetando los mejores individuos.

## 7. 📊 Trazabilidad y visualización
- Log de evolución generación por generación.
- Registro del top 3 final.
- Exportación del historial de fitness a CSV.

## 8. 🧾 Verificación final
- Conteo de horas por materia y aula.
- Reporte de materias faltantes.
- Evaluación de pares contiguos válidos insertados.

---

> 🧪 Este pipeline permite experimentar con distintas configuraciones del algoritmo y facilita la trazabilidad completa del proceso evolutivo.



